In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import os
import numpy as np
from sklearn.impute import SimpleImputer

# Plotting 
import seaborn as sns
import matplotlib.pyplot as plt

# classification models
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import Lasso
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import RidgeClassifier


# Evaluation metric
from sklearn.metrics import roc_auc_score

In [ ]:
train_df = pd.read_csv('/kaggle/input/dont-overfit-ii/train.csv')
test_df = pd.read_csv('/kaggle/input/dont-overfit-ii/test.csv')

In [ ]:
display(train_df.shape)
display(train_df.head())
display(train_df.info())

In [ ]:
display(test_df.shape)
display(test_df.head())
display(test_df.info())

In [ ]:
train_df[train_df.columns[2:]].std().hist();
plt.title('Distribution of stds of all columns');

columns have std of 1 +/- 0.1 (min and max values are 0.859, 1.15 respectively)

In [ ]:
train_df[train_df.columns[2:]].mean().hist();
plt.title('Distribution of means of all columns');

columns ave mean of 0 +/- 0.15 (min and max values are -0.2, 0.1896 respectively)

In [ ]:

display(train_df.describe())
display(test_df.describe())

In [ ]:
# train_df.groupby('target').hist(figsize=(50,50))

In [ ]:
train_df['target'].value_counts() 

Binary classification --- Disbalanced 

In [ ]:
print(train_df.isnull().any().any())
print(test_df.isnull().any().any())

NO NULLS 

In [ ]:
print(train_df.duplicated().sum()) 
print(train_df.duplicated().sum()) 

NO DUPLICATS

In [ ]:
corrs = train_df.corr().abs().unstack().sort_values(kind="quicksort").reset_index()
corrs = corrs[corrs['level_0'] != corrs['level_1']]
corrs.tail(15)

In [ ]:
# fig, ax = plt.subplots(figsize=(15,5))
# corr_matrix = train_df.corr().abs()
# sns.heatmap(corr_matrix, annot=True, linewidths=.5)

In [ ]:
Y = train_df['target']
x = train_df.drop(columns=['target'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val, y_train, y_val = train_test_split(x,Y, test_size=0.2, random_state=42,stratify=train_df['target'])

In [ ]:
y_val.shape

In [ ]:
model = Lasso(alpha=0.0299)
model1 = RidgeClassifier(alpha=0.005) # -- huge overfit 

In [ ]:
model1.fit(X_train,y_train)
ypred_train=model1.predict(X_train)
ypred_val=model1.predict(X_val)


In [ ]:
print("The train score is  = {} ".format(roc_auc_score(y_train,ypred_train)))
print("The validation score is = {}".format(roc_auc_score(y_val,ypred_val)))



In [ ]:
model.fit(X_train,y_train)
ypred_train=model.predict(X_train)
ypred_val=model.predict(X_val)

In [ ]:
ypred = model.predict(test_df)
ypred

In [ ]:
output = pd.DataFrame({'id': test_df.id,
                       'target': ypred})

output.head()

In [ ]:
output.to_csv('submission.csv', index=False)